# Step 1: Hello, Data

In [1]:
import pandas as pd

# Load primary dataset (first 500 rows)
df_primary = pd.read_csv("data/1000 Sales Records.csv").head(500)

# Load secondary metadata file
df_secondary = pd.read_csv("data/Asosmenfashion.csv")

# Display first 3 rows of primary dataset
df_primary.head(3)

,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit
0,Middle East and North Africa,Libya,Cosmetics,Offline,M,10/18/2014,686800706,10/31/2014,8446,437.20,263.33,3692591.20,2224085.18,1468506.02
1,North America,Canada,Vegetables,Online,M,11/7/2011,185941302,12/8/2011,3018,154.06,90.93,464953.08,274426.74,190526.34
2,Middle East and North Africa,Libya,Baby Food,Offline,C,10/31/2016,246222341,12/9/2016,1517,255.28,159.42,387259.76,241840.14,145419.62


## Step 2: Pick the Right Container

I considered three Python data structures: `dict`, `namedtuple`, and `set`.

- `dict` is flexible and easy to manipulate but lacks structure.
- `namedtuple` provides readable field access and immutability, which is useful for clean data rows.
- `set` is useful for uniqueness checks but not suitable for row-level data.

For this dataset, `namedtuple` would be ideal for lightweight row representation. However, since the lab requires method-based logic, I will use a custom `class` in Step 3.

## Step 3: Implement Functions and Data Structure

To structure each row of the primary dataset, I created a custom Python class called `Transaction`. This class encapsulates the key fields from each sales record and includes two methods:

- `.total()` calculates the total revenue for a transaction by multiplying `unit_price` and `units_sold`.
- `.clean()` validates that the price and quantity fields are numeric and within acceptable bounds (e.g., unit price ≤ 1000).

Using this class, I converted each row of the dataset into a `Transaction` object. This approach improves code readability and modularity, allowing me to apply transformations and filters in later steps.

In [6]:
class Transaction:
    def __init__(self, row):
        self.country = row["Country"]
        self.item_type = row["Item Type"]
        self.units_sold = row["Units Sold"]
        self.unit_price = row["Unit Price"]
        self.total_revenue = row["Total Revenue"]

    def total(self):
        return self.unit_price * self.units_sold

    def clean(self):
        return all([isinstance(self.unit_price, (int, float)),isinstance(self.units_sold, (int, float)),
            self.unit_price <= 1000
        ])

In [ ]:
transactions = [Transaction(row) for _, row in df_primary.iterrows()]

## Step 4: Bulk Loaded

We convert the primary dataset into a list of dictionaries using `df.to_dict(orient="records")`. This structure allows flexible access to each transaction and supports downstream tasks like filtering, transformation, and serialization.

Each dictionary represents a single row, making it easier to work with Python functions and custom logic outside of pandas.

In [7]:
# Convert primary dataset into list of dictionaries
records = df_primary.to_dict(orient="records")

# Display the first record
records[0]

{'Region': 'Middle East and North Africa',
 'Country': 'Libya',
 'Item Type': 'Cosmetics',
 'Sales Channel': 'Offline',
 'Order Priority': 'M',
 'Order Date': '10/18/2014',
 'Order ID': 686800706,
 'Ship Date': '10/31/2014',
 'Units Sold': 8446,
 'Unit Price': 437.2,
 'Unit Cost': 263.33,
 'Total Revenue': 3692591.2,
 'Total Cost': 2224085.18,
 'Total Profit': 1468506.02}

## Step 5: Quick Profiling

We perform basic profiling to understand the distribution and diversity of our dataset:

- **Min Unit Price**: Lowest price per unit in the dataset
- **Mean Unit Price**: Average price across all transactions
- **Max Unit Price**: Highest price per unit
- **Unique Country Count**: Number of distinct countries involved in shipping

This helps identify potential outliers and gives a sense of geographic spread.

In [8]:
# Basic price profiling
min_price = df_primary["Unit Price"].min()
mean_price = df_primary["Unit Price"].mean()
max_price = df_primary["Unit Price"].max()

# Unique city count
unique_cities = set(df_primary["Country"])
city_count = len(unique_cities)

# Display results
print("Min Unit Price:", min_price)
print("Mean Unit Price:", round(mean_price, 2))
print("Max Unit Price:", max_price)
print("Unique Country Count:", city_count)

Min Unit Price: 9.33
Mean Unit Price: 274.3
Max Unit Price: 668.27
Unique Country Count: 171


## Step 6: Spot the Grime

We examined the dataset for common data quality issues:

- **Missing Values**: Some columns contain nulls, which may affect calculations or filtering.
- **Duplicate Rows**: A few transactions are repeated, which could distort totals or aggregations.
- **Outliers**: We found unusually high `Unit Price` values (above $1000), which may be data entry errors or exceptional cases.

These issues will be addressed in the next step using cleaning rules.

In [9]:
# 1. Check for missing values
missing_summary = df_primary.isnull().sum()
print("Missing values per column:\n", missing_summary)

# 2. Check for duplicate rows
duplicate_rows = df_primary[df_primary.duplicated()]
print("Number of duplicate rows:", duplicate_rows.shape[0])

# 3. Check for outliers in Unit Price
outlier_prices = df_primary[df_primary["Unit Price"] > 1000]
print("Unit Price outliers (>1000):", outlier_prices.shape[0])

Missing values per column:
 Region            0
Country           0
Item Type         0
Sales Channel     0
Order Priority    0
Order Date        0
Order ID          0
Ship Date         0
Units Sold        0
Unit Price        0
Unit Cost         0
Total Revenue     0
Total Cost        0
Total Profit      0
dtype: int64
Number of duplicate rows: 0
Unit Price outliers (>1000): 0


## Step 7: Cleaning Rules

We applied our `.clean()` method to validate each transaction. This method checks for numeric types and reasonable price bounds.

- **Before Cleaning**: 500 rows
- **After Cleaning**: 500 rows

No rows were removed, indicating that the dataset is already clean. This confirms our earlier profiling and prepares us for safe transformations.

In [10]:
# Apply cleaning logic
cleaned_transactions = [tx for tx in transactions if tx.clean()]

# Show before/after counts
print("Before cleaning:", len(transactions))
print("After cleaning:", len(cleaned_transactions))

Before cleaning: 500
After cleaning: 500
